In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 14.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617634 sha256=7c5996a4c153f31e10cd29b28db050fe77c1bce5668a0f95d7c22ebcbad9d0e9
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
# surprise library
from surprise import Dataset, Reader

from surprise import SVD, SVDpp
from surprise.model_selection import \
    train_test_split, GridSearchCV, cross_validate, PredefinedKFold

from surprise import accuracy
from surprise.model_selection import KFold

from surprise import SlopeOne, CoClustering

from surprise import dump

import pandas as pd
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Spiderum/Data

/content/drive/MyDrive/Spiderum/Data


In [ ]:
data = pd.read_csv('./Data for CF/cleaned_data.csv')
data.head()

,user_id,redirect_url,vote_point
0,5,https://spiderum.com/bai-dang/Dau-tu-vao-nhan-...,1
1,5,https://spiderum.com/bai-dang/Toi-chi-muon-con...,1
2,8,http://spiderum.com/bai-dang/La-mot-du-hoc-sin...,1
3,8,http://spiderum.com/bai-dang/Sang-tao-va-pha-c...,1
4,8,https://spiderum.com/bai-dang/-c1l,1


# Build model SVD (Matrix Factorization)

In [ ]:
reader = Reader()
data_prepared = Dataset.load_from_df(data, reader)

In [ ]:
param_grid1 = {'n_factors':[3, 5],'n_epochs': [10, 20], 'lr_all': [0.0025, 0.005],
              'reg_all': [0.2, 0.4]}

gs_model1 = GridSearchCV(SVD, param_grid=param_grid1, return_train_measures=True)
gs_model1.fit(data_prepared)

In [ ]:
gs_model1.best_params

{'mae': {'lr_all': 0.0025, 'n_epochs': 10, 'n_factors': 3, 'reg_all': 0.4},
 'rmse': {'lr_all': 0.0025, 'n_epochs': 20, 'n_factors': 3, 'reg_all': 0.2}}

In [ ]:
param_grid2 = {'n_factors':[3, 4],'n_epochs': [15, 20], 'lr_all': [0.0025, 0.00375],
              'reg_all': [0.2, 0.3]}

gs_model2 = GridSearchCV(SVD, param_grid=param_grid2, return_train_measures=True)
gs_model2.fit(data_prepared)

In [ ]:
gs_model2.best_params

{'mae': {'lr_all': 0.0025, 'n_epochs': 15, 'n_factors': 3, 'reg_all': 0.3},
 'rmse': {'lr_all': 0.0025, 'n_epochs': 15, 'n_factors': 3, 'reg_all': 0.2}}

In [ ]:
param_grid3 = {'n_factors':[2, 3],'n_epochs': [15], 'lr_all': [0.0025, 0.001],
              'reg_all': [0.2, 0.25]}

gs_model3 = GridSearchCV(SVD, param_grid=param_grid3, return_train_measures=True)
gs_model3.fit(data_prepared)

In [ ]:
gs_model3.best_params

{'mae': {'lr_all': 0.001, 'n_epochs': 15, 'n_factors': 2, 'reg_all': 0.25},
 'rmse': {'lr_all': 0.0025, 'n_epochs': 15, 'n_factors': 2, 'reg_all': 0.2}}

In [ ]:
param_grid4 = {'n_factors':[1, 2],'n_epochs': [15], 'lr_all': [0.0025, 0.00175],
              'reg_all': [0.2, 0.1]}

gs_model4 = GridSearchCV(SVD, param_grid=param_grid4, return_train_measures=True)
gs_model4.fit(data_prepared)

In [ ]:
gs_model4.best_params

{'mae': {'lr_all': 0.00175, 'n_epochs': 15, 'n_factors': 1, 'reg_all': 0.2},
 'rmse': {'lr_all': 0.00175, 'n_epochs': 15, 'n_factors': 1, 'reg_all': 0.1}}

In [ ]:
param_grid5 = {'n_factors':[1, 10, 30, 50, 100],'n_epochs': [15], 'lr_all': [0.0025],
              'reg_all': [0.2]}

gs_model5 = GridSearchCV(SVD, param_grid=param_grid5, return_train_measures=True, cv=5)
gs_model5.fit(data_prepared)

In [ ]:
gs_model5.best_params

{'mae': {'lr_all': 0.0025, 'n_epochs': 15, 'n_factors': 1, 'reg_all': 0.2},
 'rmse': {'lr_all': 0.0025, 'n_epochs': 15, 'n_factors': 1, 'reg_all': 0.2}}

In [ ]:
gs_model5.cv_results

In [ ]:
import collections

class CF(object):
    def __init__(self, data):
        self.data = data
        self.trainset = Dataset.load_from_df(self.data, Reader()).build_full_trainset()
        self.model = None
        self.predictions = None

    def train_model(self, lr_all, n_epochs, n_factors, reg_all, biased):
        self.model = SVD(lr_all=lr_all, n_epochs=n_epochs, n_factors=n_factors, reg_all=reg_all, biased=biased)
        self.model.fit(self.trainset)
    
    def save_model(self, path):
        dump.dump('./Data for CF/svd_model',algo=self.model)
    
    def load_model(self, path):
        self.model = dump.load('./Data for CF/svd_model')[1]
    
    def create_predictions(self):
        testset = trainset.build_anti_testset()
        self.predictions = self.model.test(testset)

    def recommend_for_users(self, inner_uid_start, inner_uid_end, top_n_items):
        list_users = []
        list_items = []
        list_ratings = []
        for inner_uid in range(inner_uid_start, inner_uid_end + 1):
            inner_iid_rated_list = [inner_iid for inner_iid, rating in self.trainset.ur[inner_uid]]
            inner_iid_predict_list = list(set(self.trainset.all_items()) - set(inner_iid_rated_list))
            raw_iid_predict_list = [self.trainset.to_raw_iid(inner_iid) for inner_iid in inner_iid_predict_list]
            n = len(raw_iid_predict_list)

            list_users.extend([self.trainset.to_raw_uid(inner_uid) for _ in range(n)])
            list_items.extend(raw_iid_predict_list)
            list_ratings.extend([0 for _ in range(n)])

        df = pd.DataFrame({'users': list_users, 'items': list_items, 'ratings': list_ratings})
        predictions_set = Dataset.load_from_df(df, Reader()).build_full_trainset().build_testset()
        predictions = self.model.test(predictions_set)

        top_n = collections.defaultdict(list)
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))

        # Then sort the predictions for each user and retrieve the k highest ones.
        for uid, ratings in top_n.items():
            ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = ratings[:top_n_items]

        return top_n



In [ ]:
a.recommend_for_users(1, 10, 10)

In [ ]:
a = CF(data=data)

In [ ]:
a.train_model(lr_all=0.0025, n_epochs=15, n_factors=1, reg_all=0.2, biased=None)

In [ ]:
reader = Reader()
data_prepared = Dataset.load_from_df(data, reader)

In [ ]:
data_prepared = data_prepared.build_full_trainset()